In [3]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder

class SelectItem(BaseEstimator, TransformerMixin):
    
    def __init__(self, features=[]):
        self._features = features
    
    def fit(self, X, y=None):
        if len(self._features)==0:
            self._features = X.columns
        return self
        
    def transform(self, X, y=None):
        return X.loc[:, self._features].values
    
class SinCos(BaseEstimator, TransformerMixin):
    
    def __init__(self, dosin=True, docos=True):
        self._dosin = dosin
        self._docos = docos
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        tmp=[]
        for j in X.T:
            if self._dosin:
                tmp.append( np.sin(j) )
            if self._docos:
                tmp.append( np.cos(j) )
        return np.vstack(tmp).T
                
class Encoding(BaseEstimator, TransformerMixin):
    
    def __init__(self, drop='first', sparse=False, handle_unknown = 'ignore'):
        self._drop = drop
        self._sparse = sparse
        self._err = handle_unknown
        self._fitted = []
    
    def fit(self, X, y=None):
        for j in X.T:
            encoder = OneHotEncoder(sparse=self._sparse, handle_unknown = self._err)
            encoder.fit(j.reshape(-1,1))
            self._fitted.append(encoder)
            
        return self
    
    def transform(self, X, y=None):
        tmp=[]
        for i, j in enumerate(X.T):
            encoder = self._fitted[i]
            tmp.append( encoder.transform(j.reshape(-1,1)) )
        return np.hstack(tmp)
        

In [4]:
from sklearn.datasets import load_iris

data = load_iris()
df = pd.concat([pd.DataFrame(data.target, columns=['target']),pd.DataFrame(data.data, columns=data.feature_names)], axis=1)

In [5]:
df.head()

,target,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.1,3.5,1.4,0.2
1,0,4.9,3.0,1.4,0.2
2,0,4.7,3.2,1.3,0.2
3,0,4.6,3.1,1.5,0.2
4,0,5.0,3.6,1.4,0.2


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

pipe_sincos = Pipeline([('selsincos', SelectItem(features=['sepal width (cm)','petal length (cm)','petal width (cm)'])),
              ('sincos',  SinCos(dosin=True, docos=True) )
             ])

pipe_dum = Pipeline([('selcat', SelectItem(features=['target'])),
              ('categ',  Encoding() )
             ])
pipe_union = FeatureUnion([('etape1' ,pipe_sincos), ('etape2', pipe_dum)])

In [7]:
pipe_union.fit(df)
resultat = pipe_union.transform(df)

In [8]:
pd.DataFrame(resultat).head(2)

,0,1,2,3,4,5,6,7,8
0,-0.350783,-0.936457,0.98545,0.169967,0.198669,0.980067,1.0,0.0,0.0
1,0.141120,-0.989992,0.98545,0.169967,0.198669,0.980067,1.0,0.0,0.0
